In [ ]:
# fetch data here

In [ ]:
# extract from archives
# launch this cell from directory with archives
!for f in *.zip; do 7z x -y -mmt4 $f 1>/dev/null; done

In [ ]:
# split to folders by 100 files
import sys
import os

base = ('gtps/')
for directory in os.listdir(base):
    if os.path.isdir(os.path.join(base, directory)):
        print(directory)
        os.chdir(os.path.join(base, directory))
        os.system('ls|parallel -n100 mkdir {#}\;mv {} {#}')

In [ ]:
# make convertlists
import tqdm
import os


def proccurdir():
    with open('convertlist.txt', 'w') as outf:
        outf.write('[\n')
        is_first = True
        for root, directories, filenames in os.walk('.'):
            #print(filename)
            for filename in filenames:
                if filename[-4:].lower() != '.mid' and filename[-4:].lower() != '.zip' and filename[-4:].lower() != '.txt':
                    new_filename = filename[:-4]+'.mid'
                    path = os.path.join(os.getcwd(), root, filename).replace(r'"', r'\"')
                    new_path = os.path.join(os.getcwd(), root, new_filename).replace(r'"', r'\"')
                    if os.path.isfile(new_path):
                        continue
                    if not is_first:
                        outf.write(',')
                    else:
                        is_first = False
                    outf.write('{"in":"%s", "out":"%s"}\n'%(path, new_path))
                    #subprocess.run(["musescore", path, "-o", new_path])
                    #process_file(new_path)
                    
                else:
                    pass
        outf.write('\n]\n')
        
pb = tqdm.tqdm()

base = ('gtps/')
for directory in os.listdir(base):
    if os.path.isdir(os.path.join(base, directory)):
        pb.update(n=1)
        print(directory)
        for subdirectory in os.listdir(os.path.join(base, directory)):
            if os.path.isdir(os.path.join(base, directory, subdirectory)):
                os.chdir(os.path.join(base, directory, subdirectory))
                #print(os.path.join(base, directory, subdirectory))
                proccurdir()

In [ ]:
# converting gtp to mid
base = ('gtps/')
tqdm = tqdm.tqdm_notebook()
for directory in os.listdir(base):
    tqdm.update(n=1)
    if os.path.isdir(os.path.join(base, directory)):
        print(directory)
        for subdirectory in os.listdir(os.path.join(base, directory)):
            if os.path.isdir(os.path.join(base, directory, subdirectory)):
                os.chdir(os.path.join(base, directory, subdirectory))
                #print(os.path.join(base, directory, subdirectory))
                os.system('mscore -j convertlist.txt 2>log.txt')

In [ ]:
# launch this cell from directory with midis
!parallel --line-buffer 'cd {} && python /home/sharik/code/Innprak/accompaniator/ml/process_all.py' ::: *
